In [8]:
%matplotlib inline
%matplotlib notebook
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import motion_tracking
from random import randint
import copy

# plt.rcParams['figure.figsize'] = (15,5)
# plt.rcParams.update({'font.size': 12})

### Image Caching

In [3]:
sequence_path = "sequence"
background_img = "background.png"

# path = base path
# folders = folders within path that contain images
# Returns a list of strings of the relative path names for every image in path/ within every folder in folders
def get_files(path):
    file_list = list()
    for (dir_path, dir_names, file_names) in os.walk(path):
        file_list += [cv2.imread(os.path.join(dir_path, file)) for file in file_names]
    return file_list

images = get_files(sequence_path)  # loads and caches all the images

### Pedestrian Detection

In [4]:
class YoloDetector:
    def __init__(self, frame_width, frame_height):
        self.width = frame_width
        self.height = frame_height
        self.net = cv2.dnn.readNet('yolov3.weights','yolov3.cfg')
        with open("coco.names", "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

        layer_names = self.net.getLayerNames()
        self.output_layers = [layer_names[i[0] - 1] for i in self.net.getUnconnectedOutLayers()]
        
        #self.colors = np.random.uniform(0, 255, size=(len(classes), 3))
        #self.font = cv2.FONT_HERSHEY_PLAIN
        
    def get_detection(self, img):
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * self.width)
                    center_y = int(detection[1] * self.height)
                    w = int(detection[2] * self.width)
                    h = int(detection[3] * self.height)
                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    
                    # Retain only pedestrians
                    label = str(self.classes[class_id])
                    if label == 'person':
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
        
        # Remove overlapping duplicate boxes
        best_boxes = []
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        for index_array in indexes:
            best_boxes.append(boxes[index_array[0]])
        return best_boxes
        #return boxes, indexes, class_ids


def draw_rectangles(boxes, img, color):
    for i in range(len(boxes)):
        x, y, w, h = boxes[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        #cv2.putText(img, label, (x, y + 30), font, 3, (255,0,0), 3)
    return img

### Pedestrian Tracking

In [5]:
class Person:   
    def __init__(self, frame_width, frame_height, num_particles):
        self.bbox_history = [] # item: (x,y,w,h,frameID,isTracked)
        self.tracker = ParticleFilter(frame_width, frame_height, num_particles)
        
        
def get_optical_flow(prev_img_gray, cur_img_gray):
    height,width = cur_img_gray.shape
    hsv = np.zeros((height,width,3), dtype=cur_img_gray.dtype)
    hsv[...,1] = 255
    # optical flow (direction and velocity)
    flow = cv2.calcOpticalFlowFarneback(prev_img_gray,cur_img_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    return mag, ang, rgb, flow

def draw_flow(img, flow, step=16): # flow lines in direction of motion
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

### Runtime

In [17]:
# setup
height, width, channels = images[0].shape
detector = YoloDetector(width, height)
DETECT_WINDOW = 'Detection'
FLOW_WINDOW = 'Flow'

for n, cur_img in enumerate(images[1:]):
    # setup
    cur_img_gray = cv2.cvtColor(cur_img,cv2.COLOR_BGR2GRAY)
    
    # 1. apply detector on new frame
    detected_boxes = detector.get_detection(cur_img)

    # 2. match detections to existing People
    if n > 0:
        # **** compute bbox velocities (pairs of frames)
        flow_mag, flow_ang, flow_rgb, flow_res = get_optical_flow(prev_img_gray, cur_img_gray)
        flow_arrow = draw_flow(cur_img_gray, flow_res, step=32)
        # **** update predictions
        # **** match prediction to closest bbox (use tracker if none found)
        # **** update+resample trackers
    # 3. create new People for unmatched bboxes
    
    # draw results
    cur_img_display = copy.copy(cur_img)
    draw_rectangles(detected_boxes, cur_img_display, (255,0,0))
    cv2.putText(cur_img_display, f'Fm {n}', (0,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0),2);
    
    cv2.imshow(DETECT_WINDOW, cur_img_display)
    
    if n > 0:
        cv2.imshow(FLOW_WINDOW, flow_arrow)
    else:
        cv2.imshow(FLOW_WINDOW, cur_img_display)
    
    # setup for next iteration
    prev_img_gray = cur_img_gray
    
    # early exit
    ch = cv2.waitKey(5) # millis delay
    if ch == 27: # press ESC on the main window to exit
        cv2.destroyAllWindows()
        break

In [ ]:
flow_mag, flow_ang, flow_rgb, flow_res = get_optical_flow(prev_img_gray, cur_img_gray)

In [ ]:
flow_res.shape

In [ ]:
max(flow_res.ravel())